In [1]:
import pyspark

myConf=pyspark.SparkConf()
spark= pyspark.sql.SparkSession\
    .builder\
    .master('local')\
    .appName('myApp')\
    .config(conf=myConf)\
    .getOrCreate()

1) DataFrame을 생성

In [2]:
import os
fname= os.path.join('data', '제74주년_경찰의_날_기념식_축사.txt')
myDf= spark.read.text(fname)
myDf= myDf.select(myDf['value'].alias('sent'))
myDf.show(4)

+---------------------------------+
|                             sent|
+---------------------------------+
| 존경하는 국민 여러분, 경찰관 ...|
|                                 |
|국민의 안전을 위해 밤낮없이 애...|
|                                 |
+---------------------------------+
only showing top 4 rows



In [3]:
# 공백인 행이 포함되어 있으므로 빈 행을 제거
import pandas as pd
import numpy as np

_myDf= myDf.toPandas()
_myDf['sent'].replace('', np.nan, inplace=True)

In [4]:
_myDf.dropna(subset=['sent'], inplace=True)

In [5]:
myDf= spark.createDataFrame(_myDf)

In [6]:
myDf.show(4)

+----------------------------------+
|                              sent|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
| 국민의 안전을 위해 밤낮없이 애...|
|오늘 홍조근정훈장을 받으신 중앙...|
|           사랑하는 경찰관 여러분,|
+----------------------------------+
only showing top 4 rows



2) 단어로 분리해서, 출력

In [7]:
from pyspark.ml.feature import RegexTokenizer
regtokenizer= RegexTokenizer(inputCol='sent', outputCol='words', pattern='\\s+')

In [8]:
tokDf= regtokenizer.transform(myDf)

In [9]:
tokDf.show(4)

+----------------------------------+--------------------------------+
|                              sent|                           words|
+----------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|
+----------------------------------+--------------------------------+
only showing top 4 rows



 3) 불용어 구성, 출력

In [10]:
from pyspark.ml.feature import StopWordsRemover
stopwords=[u'돌', u'참', u'더', u'될', u'그', u'등', u'법', u'큰', u'수'] #'경찰의 날'의 '날'에는 따옴표가 붙어있어 한 글자로 카운트X

In [11]:
stop= StopWordsRemover(inputCol='words', outputCol='nostops')
stop.setStopWords(stopwords)

StopWordsRemover_9f14a9442964

In [12]:
for e in stop.getStopWords():
    print(e, end='/')

돌/참/더/될/그/등/법/큰/수/

4) 불용어 제거하고, 출력

In [13]:
stopDf= stop.transform(tokDf)
for e in stopDf.select('words', 'nostops').take(4):
    print(e)
    print()

Row(words=['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '돌', '‘경찰의', '날’입니다.'], nostops=['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '‘경찰의', '날’입니다.'])

Row(words=['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.'], nostops=['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.'])

Row(words=['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.'], nostops=['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.'])

Row(words=['사랑하는', '경찰관', 

5) TF-IDF를 계산하고, 출력

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(stop_words=stopwords, norm=None)

In [15]:
doc= myDf.rdd.flatMap(lambda line: line).collect()

In [16]:
vectorizer.fit_transform(doc)

<21x347 sparse matrix of type '<class 'numpy.float64'>'
	with 404 stored elements in Compressed Sparse Row format>

In [17]:
doc_tf= vectorizer.vocabulary_
print(doc_tf)

{'존경하는': 278, '국민': 80, '여러분': 212, '경찰관': 45, '일흔네': 251, '경찰의': 53, '입니다': 257, '국민의': 84, '안전을': 201, '위해': 232, '밤낮없이': 165, '애쓰시는': 206, '전국의': 266, '15만': 1, '여러분께': 213, '먼저': 144, '감사를': 15, '드립니다': 134, '전몰': 267, '순직': 194, '경찰관들의': 46, '고귀한': 59, '희생에': 346, '경의를': 42, '표합니다': 325, '유가족': 233, '위로의': 228, '마음을': 139, '전합니다': 270, '오늘': 220, '홍조근정훈장을': 340, '받으신': 159, '중앙경찰학교장': 290, '이은정': 243, '치안감님': 315, '근정포장을': 96, '광주남부경찰서': 75, '김동현': 103, '경감님을': 38, '비롯한': 175, '수상자': 192, '각별한': 14, '축하와': 309, '또한': 137, '경찰': 43, '영웅으로': 219, '추서되신': 308, '차일혁': 300, '최중락님께': 307, '사랑을': 178, '전해드립니다': 271, '사랑하는': 179, '여러분의': 215, '헌신적': 338, '노력으로': 112, '우리의': 227, '치안은': 317, '좋아졌습니다': 280, '지난해': 292, '범죄': 167, '발생은': 161, '2015년에': 5, '비해': 176, '15': 0, '줄었습니다': 288, '같은': 21, '기간': 97, '교통사고': 76, '사망자는': 180, '18': 2, '감소했습니다': 18, '치안의': 319, '개선은': 23, '체감으로': 304, '나타나고': 108, '있습니다': 259, '올해': 221, '상반기': 182, '체감안전도는': 303, '74': 9, '5점으로': 8, '역대': 216, '최고를': 

In [18]:
doc_idf= vectorizer.idf_
print(doc_idf)

[3.39789527 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527
 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527
 3.39789527 2.99243016 3.39789527 2.99243016 3.39789527 2.99243016
 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527
 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527
 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527
 2.70474809 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527
 3.39789527 2.70474809 3.39789527 2.29928298 3.39789527 3.39789527
 3.39789527 3.39789527 3.39789527 2.1451323  2.99243016 2.29928298
 2.99243016 3.39789527 3.39789527 2.99243016 3.39789527 3.39789527
 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527
 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527 3.39789527
 3.39789527 3.39789527 2.99243016 3.39789527 3.39789527 3.39789527
 3.39789527 3.39789527 3.39789527 2.99243016 3.39789527 3.39789527
 2.29928298 3.39789527 3.39789527 3.39789527 3.39789527 3.3978

6) TF-IDF 컬럼을 features로 구성, 출력

In [20]:
import pandas as pd
idfDf= pd.DataFrame(list(doc_tf.items()), columns=['word', 'ID'])

In [21]:
idfDf=idfDf.assign(idf=doc_idf)

In [22]:
df6= spark.createDataFrame(idfDf)

In [23]:
df6.show(4)

+--------+---+------------------+
|    word| ID|               idf|
+--------+---+------------------+
|존경하는|278|3.3978952727983707|
|    국민| 80|3.3978952727983707|
|  여러분|212|3.3978952727983707|
|  경찰관| 45|3.3978952727983707|
+--------+---+------------------+
only showing top 4 rows

